In [ ]:
from typing import Iterator, AsyncIterator, List
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document
from pdf2image import convert_from_path
from openai import OpenAI
import tempfile
import base64

client = OpenAI()

class MultimodalLoader(BaseLoader):
    def __init__(self, file_path: str, model: str = "gpt-5-nano", first_page: int = None, last_page: int = None) -> None:

        self.file_path = file_path
        self.model = model
        self.first_page = first_page
        self.last_page = last_page

    def _image_to_text(self, image) -> str:
        """단일 이미지에서 텍스트 추출 (동기)"""
        with tempfile.NamedTemporaryFile(suffix=".png") as tmp:
            image.save(tmp.name, format="PNG")
            with open(tmp.name, "rb") as f:
                b64_image = base64.b64encode(f.read()).decode("utf-8")
                response = client.responses.create(
                    model=self.model,
                    input=[
                        {"role": "developer", "content": '''You are a multimodal OCR and document parsing assistant. 
Your task is to extract text from images of PDF pages and return it in clean, structured Markdown format.

### Instructions:
1. Read the provided image(s) of the PDF page carefully. 
2. Extract all visible text, preserving the original reading order.
3. Use **Markdown syntax** for structure:
   - Use `#`, `##`, `###` for headings (follow the visual hierarchy of the document).
   - Use bullet points (`-` or `*`) and numbered lists where appropriate.
   - Use code blocks (```) only if the document contains actual code or fixed-width text.
   - Use tables in Markdown format if the document contains tabular data.
4. Do **not** insert hallucinated content. Only output text that is actually present in the image.
5. Correct minor OCR errors if possible, but do not paraphrase or rewrite the content.
6. If an image has no readable text, return:  

[No readable text found]

### Output:
Return only the extracted content in Markdown, without additional explanations.'''},
                        {"role": "user", "content": [
                            {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image}"}
                        ]}
                    ],
                )
        return response.output_text

    def lazy_load(self) -> Iterator[Document]:
        """동기식 lazy loader"""
        pages = convert_from_path(
            self.file_path,
            dpi=512,
            first_page=self.first_page,
            last_page=self.last_page
            )
        total_pages = len(pages)
        for page_num, image in enumerate(pages, start=1):
            text = self._image_to_text(image)
            yield Document(
                page_content=text,
                metadata={"source": self.file_path, "page": page_num, "total_pages": total_pages},
            )

In [6]:
loader = MultimodalLoader(
    "../../data/raw/[삼성전자]분기보고서(2025.05.15).pdf",
    model="gpt-5-mini",
    )

In [ ]:
import json
import os

output_file = "../../data/parsed/multimodal.jsonl"

if not os.path.exists(output_file):
    open(output_file, "w", encoding="utf-8").close()

with open(output_file, "a", encoding="utf-8") as f:
    for doc in loader.lazy_load():
        record = doc.model_dump()
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

In [6]:
import json
from langchain_core.documents import Document

output_file = "../../data/parsed/multimodal.jsonl"

docs = []
with open(output_file, "r", encoding="utf-8") as f:
    for line in f:
        d = json.loads(line) 
        docs.append(Document(**d))